In [270]:
import os

In [271]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'

In [272]:
os.chdir("../")

In [273]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml'

In [274]:
proj_link = 'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'
os.chdir(proj_link)


In [275]:
# create root entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    preprocessor_name: str
    label_encoder_names: str
    sm_label_encoder: str
    sm_model_name: str
    sm_processor_name: str
    target_column: str
    random_state: int
    max_depth: int
    n_estimators: int
    sm_model_pipeline_name: str
    rf_model_name: str
    rf_preprocessor_name: str
    xgb_model_name: str
    xgb_preprocessor_name: str
    xgb_selected: str
    xgb_pipeline: str
   

    

In [276]:
proj_link = 'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'
os.chdir(proj_link)

In [277]:
#create configuration manager 
#from clientClassifier.constants import *
from clientClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from clientClassifier.utils.common import read_yaml, create_directories

In [278]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer     
        params = self.params.xgb_classifier_best
        schema = self.schema.TARGET_COLUMN
        
        

        create_directories([config["root_dir"]])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            preprocessor_name=config.preprocessor_name,
            sm_model_pipeline_name=config.sm_model_pipeline_name,
            label_encoder_names=config.label_encoder_names,
            sm_label_encoder=config.sm_label_encoder,
            sm_model_name=config.sm_model_name,
            sm_processor_name=config.sm_processor_name,
            target_column=schema.name,
            max_depth = params.max_depth,
            #class_weight=params.class_weight,
            n_estimators= params.n_estimators,
            random_state=params.random_state,
            rf_model_name   = config.rf_model_name,
            rf_preprocessor_name=config.rf_preprocessor_name,
            xgb_model_name=config.xgb_model_name,
            xgb_preprocessor_name=config.xgb_preprocessor_name,
            xgb_selected=config.xgb_selected,
            xgb_pipeline=config.xgb_pipeline
           
        )

        return model_trainer_config


In [279]:
#define model trainer components
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
import joblib
import matplotlib.pyplot as plt
from clientClassifier import logger
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE 
from imblearn.pipeline import Pipeline as ImbPipeline   
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np
import optuna




In [ ]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    #def train(self):
        #logger.info("Training model")

        # Load train and test data
        #train_data = pd.read_csv(self.config.train_data_path)
        #test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target variable
        #X_train = train_data.drop(columns=[self.config.target_column], axis=1)
        #y_train = train_data[self.config.target_column]

        #X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        #y_test = test_data[self.config.target_column]

        # Identify numerical and categorical feature columns
        #numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        #categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

        # Apply Label Encoding to target variable
        #label_encoder = LabelEncoder()
        #y_train = label_encoder.fit_transform(y_train)
        #y_test = label_encoder.transform(y_test)

        # Preprocessing for features
        #numerical_transformer = StandardScaler()
        #categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        #preprocessor = ColumnTransformer(
            #transformers=[
                #('num', numerical_transformer, numerical_features),
                #('cat', categorical_transformer, categorical_features)
            #]
        #)

        # Now you can fit a model using preprocessor and encoded targets

        #logger.info("Preprocessing and label encoding complete")

        
        # create a pipeline with preprocessing and model
        #pipeline = Pipeline([   
            #('preprocessor', preprocessor),
            #('classifier', LogisticRegression(
                #class_weight=self.config.class_weight,
                #max_iter=self.config.max_iter,
                #penalty=self.config.penalty,
                #C=self.config.C,
                #solver = self.config.solver,
                #random_state=self.config.random_state
            #))
        #])      
        
        # fit the pipeline on the training data
        #pipeline.fit(X_train, y_train)      
        
     
        # make predictions on the test data 
        #y_pred = pipeline.predict(X_test)      
         
        # evaluate the model
        #report = classification_report(y_test, y_pred)
        #cm = confusion_matrix(y_test, y_pred)   
        #accuracy = accuracy_score(y_test, y_pred)   
        
        #create Confusion Matrix Display
        #disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
        #disp.plot()
        #plt.title("Logistic Regression Matrix without SMOTE")
        
        
        
        
                    

        #logger.info(f"Classification Report:\n{report}")
        #logger.info(f"Confusion Matrix:\n{cm}") 
        #logger.info(f"Accuracy: {accuracy}")   
         
        # save the model    
        
        
        #model = pipeline.named_steps['classifier']
        #preprocessor = pipeline.named_steps['preprocessor'] 

        #model_path = os.path.join(self.config.root_dir, self.config.model_name)                     
        #preprocessor_path = os.path.join(self.config.root_dir, self.config.preprocessor_name)
        #joblib.dump(preprocessor, preprocessor_path)    
        #joblib.dump(model, model_path)
        #joblib.dump(label_encoder, os.path.join(self.config.root_dir, self.config.label_encoder_names))
          
        
        #logger.info(f"Model saved at: {model_path}")
        #logger.info(f"Preprocessor saved at: {preprocessor_path}")      
        
     
        
  
    #def train_with_SMOTE(self):
        #logger.info("Training model with SMOTE")

        # Load data
        #train_data = pd.read_csv(self.config.train_data_path)
        #test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        #X_train = train_data.drop(columns=[self.config.target_column])
        #y_train = train_data[self.config.target_column]

        #X_test = test_data.drop(columns=[self.config.target_column])
        #y_test = test_data[self.config.target_column]
        
        # Apply Label Encoding to target variable
        #label_encoder = LabelEncoder()
        #y_train = label_encoder.fit_transform(y_train)
        #y_test = label_encoder.transform(y_test)

        # Define numerical and categorical features
        #numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        #categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
        
   
        # Transformers
        #numerical_transformer = StandardScaler()
        #categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        #sm_preprocessor = ColumnTransformer(
            #transformers=[
                #('num', numerical_transformer, numerical_features),
                #('cat', categorical_transformer, categorical_features)
            #]
        #)

        #  Fitting model on resampled data
        #model = LogisticRegression(
            #class_weight=self.config.class_weight,
            #max_iter=self.config.max_iter,
            #penalty=self.config.penalty,
            #C=self.config.C,
            #solver = self.config.solver,
            #random_state=self.config.random_state
        #)
        
        # use the sm_pipeline to transform the data
        #sm_pipeline = ImbPipeline([
            #('preprocessor', sm_preprocessor),
            #('smote', SMOTE(random_state=self.config.random_state)),
            #('classifier', model)
        #])

        #sm_pipeline.fit(X_train, y_train)
            
        # make predictions on the test data 
        #y_pred = sm_pipeline.predict(X_test)

        # evaluate the model
        #resampled_report = classification_report(y_test, y_pred)
        #resampled_cm = confusion_matrix(y_test, y_pred)   
        #resampled_accuracy = accuracy_score(y_test, y_pred)
        
        #create Confusion Matrix Display
        #cm_display = ConfusionMatrixDisplay(confusion_matrix=resampled_cm, display_labels=label_encoder.classes_)
        #cm_display.plot()
        #plt.title("Logistic Regression with Matrix SMOTE")
                                            

        #logger.info(f"Classification Report:\n{resampled_report}")
        #logger.info(f"Confusion Matrix:\n{resampled_cm}") 
        #logger.info(f"Accuracy: {resampled_accuracy}")   
            
    
        # Save the model and preprocessor
        #joblib.dump(sm_pipeline,
                    #os.path.join(self.config.root_dir, 
                    #self.config.sm_model_pipeline_name))
        
       
         

        
        
        
    #def train_RandomForest(self):
        #logger.info("Training Random Forest model")

        # Load data
        #train_data = pd.read_csv(self.config.train_data_path)
        #test_data = pd.read_csv(self.config.test_data_path)

         # Separate features and target
        #X_train = train_data.drop(columns=[self.config.target_column])
        #y_train = train_data[self.config.target_column]

        #X_test = test_data.drop(columns=[self.config.target_column])
        #y_test = test_data[self.config.target_column]
        
        
        # Apply Label Encoding to target variable
        #label_encoder = LabelEncoder()
        #y_train = label_encoder.fit_transform(y_train)
        #y_test = label_encoder.transform(y_test)

         # Define numerical and categorical features
        #numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        #categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

        # Transformers
        #numerical_transformer = StandardScaler()
        #categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        #rf_preprocessor = ColumnTransformer(
            #transformers=[
                #('num', numerical_transformer, numerical_features),
                #('cat', categorical_transformer, categorical_features)
            #]
        #)


        # Preprocess X_train using fit_transform (fit only on training data)
        #X_train_processed = rf_preprocessor.fit_transform(X_train, y_train)

        # Fitting Random Forest model on the processed data
        #rf_model = RandomForestClassifier(max_depth= self.config.max_depth,
                                          #n_estimators=self.config.n_estimators, random_state=self.config.random_state,
                                          #class_weight=self.config.class_weight ) 
        



    
        #rf_model.fit(X_train_processed, y_train)        
        # make predictions on the test data 
        #X_test_processed = rf_preprocessor.transform(X_test)
        #y_pred_rf = rf_model.predict(X_test_processed)

        # evaluate the model
        #rf_report = classification_report(y_test, y_pred_rf)
        #rf_cm = confusion_matrix(y_test, y_pred_rf)   
        #rf_accuracy = accuracy_score(y_test, y_pred_rf)   

        #logger.info(f"Random Forest Classification Report:\n{rf_report}")
        #logger.info(f"Random Forest Confusion Matrix:\n{rf_cm}") 
        #logger.info(f"Random Forest Accuracy: {rf_accuracy}")
        
        # save the model
        #rf_model_path = os.path.join(self.config.root_dir, self.config.rf_model_name)   
        #rf_preprocessor_path = os.path.join(self.config.root_dir, self.config.rf_preprocessor_name)
        #joblib.dump(rf_model, rf_model_path)
        #joblib.dump(rf_preprocessor, rf_preprocessor_path)
        #return rf_report, rf_cm, rf_accuracy    
   
    
    
    def train_XGBClassifier(self):
        logger.info("Training XGBoost model")

        # Load data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]

        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]
        
        
        # Apply Label Encoding to target variable
        label_encoder = LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
        y_test = label_encoder.transform(y_test)
        
    

        # calculating "scale_pos_weight"
        neg = np.sum(y_train == 0)  # count of negative class
        pos = np.sum(y_train == 1)  # count of positive class

        scale_pos_weight = neg / pos
        print(f"scale_pos_weight = {scale_pos_weight:.2f}")
        
        
        categorical_features = ['month', 'poutcome']

        # Define numerical and categorical features
        numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
        
        

        # Transformers
        numerical_transformer = StandardScaler()
        categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        xgb_preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

        # Preprocess X_train using fit_transform (fit only on training data)
        X_train_processed = xgb_preprocessor.fit_transform(X_train)

        # Fitting XGBoost model on the processed data
        xgb_model = XGBClassifier(random_state=self.config.random_state,
                                   n_estimators=self.config.n_estimators,
                                   max_depth=self.config.max_depth,
                                   scale_pos_weight=scale_pos_weight)
        
          
        xgb_model.fit(X_train_processed, y_train)

        # make predictions on the test data 
        X_test_processed = xgb_preprocessor.transform(X_test)
        
        
        
        y_pred_xgb = xgb_model.predict(X_test_processed)

        # evaluate the model
        xgb_report = classification_report(y_test, y_pred_xgb)
        xgb_cm = confusion_matrix(y_test, y_pred_xgb)   
        xgb_accuracy = accuracy_score(y_test, y_pred_xgb)   
        
        #create Confusion Matrix Display
        cm_display = ConfusionMatrixDisplay(confusion_matrix=xgb_cm, display_labels=label_encoder.classes_)
        cm_display.plot()
        plt.title("XGBClassifier Matrix")

        logger.info(f"XGBoost Classification Report:\n{xgb_report}")
        logger.info(f"XGBoost Confusion Matrix:\n{xgb_cm}") 
        logger.info(f"XGBoost Accuracy: {xgb_accuracy}")
        
        # save the model
        xgb_label_path = os.path.join(self.config.root_dir, self.config.label_encoder_names )
        xgb_model_path = os.path.join(self.config.root_dir, self.config.xgb_model_name)   
        xgb_preprocessor_path = os.path.join(self.config.root_dir, self.config.xgb_preprocessor_name)
        
        joblib.dump(label_encoder, xgb_label_path)
        joblib.dump(xgb_model, xgb_model_path)
        joblib.dump(xgb_preprocessor, xgb_preprocessor_path)
        return xgb_report, xgb_cm, xgb_accuracy
    
        
            


    def train_selected_features(self):
        logger.info("Training XGBClassifier model on selected features")

        # Load data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]
        
        
        
        important_features = ['age', 'month', 'day', 'balance', 'poutcome']
        X_train = X_train[important_features]

        #X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]
        
        # Label Encoding
        label_encoder = LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
        y_test = label_encoder.transform(y_test)
        
        # Scale Pos Weight
        neg = np.sum(y_train == 0)
        pos = np.sum(y_train == 1)
        scale_pos_weight = neg / pos
        print(f"scale_pos_weight = {scale_pos_weight:.2f}")
        
        
        # Convert month to string
        X_train["month"] = X_train["month"].astype(str)
        
    
        # Preprocessing
        numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
        
        print(f"Numerical features: {numerical_features}")
        print(f"Categorical features: {categorical_features}")

        numerical_transformer = StandardScaler()
        categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        xgb_preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

        # Preprocess
        X_train_processed = xgb_preprocessor.fit_transform(X_train)
        
        print(X_train_processed[20:30])

        #  Define Optuna objective function
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'max_depth': trial.suggest_int('max_depth', 2, 20),
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'gamma': trial.suggest_float('gamma', 0, 5),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
                'scale_pos_weight': scale_pos_weight,
                'random_state': self.config.random_state,
                
            }
            
            model = XGBClassifier(**params)
            model.fit(X_train_processed, y_train)

            preds = model.predict(X_train_processed)  
            acc = accuracy_score(y_train, preds)
            
            return acc  # maximized accuracy

        # Create Optuna study
        study = optuna.create_study(direction="maximize", study_name="xgb_selected_features_optimization")
        study.optimize(objective, n_trials=30)  

        print("Best parameters:", study.best_params)

        # Train final model with best parameters
        best_params = study.best_params
        best_params.update({
            'scale_pos_weight': scale_pos_weight,
            'random_state': self.config.random_state,
           
            'eval_metric': 'logloss'
        })

        # Define full pipeline with preprocessor and classifier
        final_pipeline = Pipeline([
            ('preprocessor', xgb_preprocessor),
            ('classifier', XGBClassifier(**best_params))
        ])

        # Fit the pipeline on raw X_train
        final_pipeline.fit(X_train, y_train)
                
        # Save everything
        xgb_label_path = os.path.join(self.config.root_dir, self.config.label_encoder_names )
        xgb_model_path = os.path.join(self.config.root_dir, self.config.xgb_pipeline)
        xgb_processor_path= os.path.join(self.config.root_dir, self.config.xgb_preprocessor_name)  
  

        joblib.dump(label_encoder, xgb_label_path)
        joblib.dump(final_pipeline, xgb_model_path)
        joblib.dump(xgb_preprocessor, xgb_processor_path)
    

        logger.info("Training completed and artifacts saved!")



In [281]:


try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    #model_trainer.train()
    #model_trainer.train_with_SMOTE() 
    #model_trainer.train_RandomForest()

    model_trainer.train_selected_features()
except Exception as e:
    logger.exception(e)
    raise e 



[2025-05-03 18:11:44,828: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-03 18:11:44,844: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-03 18:11:44,859: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-03 18:11:44,864: INFO: common: created directory at: artifacts]
[2025-05-03 18:11:44,867: INFO: common: created directory at: artifacts/model_trainer]
[2025-05-03 18:11:44,871: INFO: 3206532855: Training XGBClassifier model on selected features]


[I 2025-05-03 18:11:44,996] A new study created in memory with name: xgb_selected_features_optimization


scale_pos_weight = 7.55
Numerical features: ['age', 'day', 'balance']
Categorical features: ['month', 'poutcome']
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 50 stored elements and shape (10, 19)>
  Coords	Values
  (0, 0)	0.5414148720797152
  (0, 1)	0.004497571507274968
  (0, 2)	0.42350069524963224
  (0, 10)	1.0
  (0, 18)	1.0
  (1, 0)	-0.2092225312043494
  (1, 1)	0.004497571507274968
  (1, 2)	-0.3696977005191168
  (1, 10)	1.0
  (1, 18)	1.0
  (2, 0)	1.2920522753637798
  (2, 1)	-0.9664408506005748
  (2, 2)	-0.7598423002462411
  (2, 10)	1.0
  (2, 18)	1.0
  (3, 0)	-0.7722005836673977
  (3, 1)	-1.2091754561275374
  (3, 2)	0.8204584819205207
  (3, 11)	1.0
  (3, 18)	1.0
  (4, 0)	-0.9598599344884139
  (4, 1)	-0.8450735478370937
  (4, 2)	3.9993480743662535
  (4, 14)	1.0
  (4, 15)	1.0
  (5, 0)	1.1043929245427635
  (5, 1)	-1.2091754561275374
  (5, 2)	-0.4987896636641212
  (5, 10)	1.0
  (5, 18)	1.0
  (6, 0)	-0.4907115574358736
  (6, 1)	-0.6023389423101312
  (6, 2)	-0.504527084248

[I 2025-05-03 18:11:46,910] Trial 0 finished with value: 0.9455199115044248 and parameters: {'n_estimators': 292, 'max_depth': 18, 'learning_rate': 0.025874941016387164, 'subsample': 0.6167333942810858, 'colsample_bytree': 0.5674338173447957, 'gamma': 0.08810476876420448, 'reg_alpha': 2.051873654065613, 'reg_lambda': 2.935014676941164}. Best is trial 0 with value: 0.9455199115044248.
[I 2025-05-03 18:11:48,043] Trial 1 finished with value: 0.8962942477876106 and parameters: {'n_estimators': 312, 'max_depth': 15, 'learning_rate': 0.043022621049036834, 'subsample': 0.7793828929115905, 'colsample_bytree': 0.9629954626536317, 'gamma': 2.306054069000843, 'reg_alpha': 4.020577711048117, 'reg_lambda': 2.541808385531051}. Best is trial 0 with value: 0.9455199115044248.
[I 2025-05-03 18:11:49,553] Trial 2 finished with value: 0.8722345132743363 and parameters: {'n_estimators': 479, 'max_depth': 7, 'learning_rate': 0.007991974017894875, 'subsample': 0.8474291405571421, 'colsample_bytree': 0.9698

Best parameters: {'n_estimators': 437, 'max_depth': 11, 'learning_rate': 0.19540432243618622, 'subsample': 0.8390273159611416, 'colsample_bytree': 0.8261737210662714, 'gamma': 0.10573531308274864, 'reg_alpha': 0.4512250599335613, 'reg_lambda': 2.1078300426687555}
[2025-05-03 18:12:17,240: INFO: 3206532855: Training completed and artifacts saved!]
